## Open Government Data, provided by **OpenDataZurich**
*Autogenerated Python starter code for data set with identifier* **geo_liste_der_informationstafeln_an_denkmalschutz_objekten**

## Dataset
# **Liste der Informationstafeln an Denkmalschutz-Objekten**

## Description

Der Datensatz enthält die Adressen, textlichen Beschreibungen und die Standorte der blauen Informationstafeln an kunst- und kulturhistorischen Schutzobjekten in der Stadt Zürich.
Die Tafeln können an Gebäuden, Gebäudeteilen, Brunnen, Brücken usw. angebracht sein. Voraussetzung für das Anbringen einer blauen Tafel ist in der Regel die formelle Unterschutzstellung des Objekts.

Die Denkmalschutz-Tafeln beziehen sich auf ein oder mehrere kunst- und kulturhistorischen Schutzobjekte und sind teilweise auch an Nicht-Inventar-Objekten angebracht. Manchmal sind auch mehrere Tafeln an einem Objekt montiert. Es besteht daher keine 1:1 Beziehung zum Datensatz des Denkmalinventars. Auch die Georeferenzierung erfolgte unabhängig und nach unterschiedlichen Gesichtspunkten. Der Standort der Tafeln wurde nur in einzelnen Fällen präzise erfasst, oft wurde eine von mehreren Adressen des beschriebenen Objekts verwendet.
Eine spätere logische Zusammenführung mit dem Denkmalinventar ist denkbar.  

**Zweck**: Veröffentlichung einer Gesamtübersicht zu den bestehenden Denkmalschutztafeln in der Stadt Zürich.
Arbeitsinstrument im Bereich Archäologie &amp; Denkmalpflege des Amts für Städtebau (AfS).

**Genereller Hinweis zum Geodatensatz:**


Weitere Informationen zu diesem Datensatz finden sie unter [«**Liste der Informationstafeln an Denkmalschutz-Objekten»** auf Geocat.ch](https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/79c64441-32d5-432f-b284-a66f4cea972a).				
			   

**Informationen zum Datensatz:**

Die dunkelblau emaillierten quadratischen Tafeln wurden zur Auszeichnung von denkmalgeschützten Bauten 1975 anlässlich des Europäischen Jahrs für Denkmalpflege und Heimatschutz vom damaligen 'Büro für Denkmalpflege' der Stadt Zürich eingeführt. Zur Zeit der Erstveröffentlichung im März 2021 sind 655 Tafeln vorhanden, wobei nicht ganz alle zu diesem Zeitpunkt auch montiert sind.
Obschon die Liste regelmässig nachgeführt wird, kann nicht ausgeschlossen werden, dass in Einzelfällen Abweichungen bezüglich Inhalt und Umfang von den tatsächlich montierten Tafeln bestehen.
Hinweis: Die Online-Veröffentlichung der Liste der mit blauen Tafeln versehenen kunst- und kulturhistorischen Schutzobjekte der Stadt Zürich hat nicht die rechtlichen Folgen wie eine Inventareröffnung nach § 209 PBG. Insbesondere werden keinerlei Fristen und keine entsprechenden Rechtsfolgen oder irgendwelche Handlungspflichten der Gemeinde ausgelöst. Die Liste hat rein informativen Charakter im Sinne einer Öffentlichkeitsarbeit. 

**Datenerfassung:**

Die Liste der blauen Tafeln wird im Amt für Städtebau (AfS) vom Fachbereich Inventarisation Denkmalpflege geführt. Änderungen ergeben sich, wenn zusätzliche Tafeln erstellt und am Objekt montiert werden.

**Datengrundlage:**

Grundlage sind die beschreibenden Informationstexte mit Adresse und Georeferenzierung, die für eine Auswahl von kunst- und kulturhistorischen Schutzobjekten in der Stadt Zürich erstellt und in Form von blauen Tafeln durch die Stadt Zürich an den Objekten angebracht wurden. 

**Statisches Vorschaubild:**

![BildText](https://www.gis.stadt-zuerich.ch/zueriplan_docs/geocat/79c64441-32d5-432f-b284-a66f4cea972a.png)



## Data set links

[Direct link by OpenDataZurich for dataset](https://data.stadt-zuerich.ch/dataset/geo_liste_der_informationstafeln_an_denkmalschutz_objekten)

https://www.stadt-zuerich.ch/geodaten/download/Liste_der_Informationstafeln_an_Denkmalschutz_Objekten?format=geojson_link<br>


## Metadata
- **Publisher** `Archäologie & Denkmalpflege, Amt für Städtebau, Hochbaudepartement`
- **Maintainer** `Open Data Zürich`
- **Maintainer_email** `opendata@zuerich.ch`
- **Keywords** `Bauen und Wohnen,Bevölk­erung`
- **Tags** `['denkmalpflege', 'denkmalschutz', 'geodaten', 'geoportal', 'geschutzte-bauten', 'heimatschutz', 'inventar', 'punktdaten', 'schutzobjekte', 'stzh']`
- **Metadata_created** `2023-11-06T03:08:42.471363`
- **Metadata_modified** `2026-02-14T06:31:55.773923`


## Imports and helper functions

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd 

import json
import xml.etree.ElementTree as ET
import requests
import geopandas as gpd
import re

In [ ]:
# helper function for reading datasets with proper separator
def get_dataset(url):
    # get URL ofs WFS service
    dataset_identifier = re.search(r'\/([^\/\?]+)\?', url).group(1)
    url_geoportal = f"https://www.ogd.stadt-zuerich.ch/wfs/geoportal/{dataset_identifier}"
    print("Getting available layers from:", url_geoportal)
    
    # Parameter for GetCapabilities
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetCapabilities"
    }
    
    # send GetCapabilities
    response = requests.get(url_geoportal, params=params)

    # parse XML answer
    root = ET.fromstring(response.content)

    # define Namespace 
    namespace = {'wfs': 'http://www.opengis.net/wfs'}
    
    # Exctract available layers
    layers = [feature_type.find('wfs:Name', namespace).text for feature_type in root.findall('.//wfs:FeatureType', namespace)]
    
    print("Available layers:", layers)
    print("First layer is set as default. To chose another layer set it as typename in the get_dataset() function.")

    # set first layer als typename
    typename = layers[0]
    print("Chosen typename:", typename)

    # Parameter GetFeature request
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetFeature",
        "typename": typename,
        "outputFormat": "application/json"
    }
    
    # GetFeature request
    response = requests.get(url_geoportal, params=params)

    # Load GeoJSON in GeoDataFrame
    gdf = gpd.read_file(json.dumps(response.json()))
    return gdf

## Load the data

In [ ]:
gdf = get_dataset('https://www.stadt-zuerich.ch/geodaten/download/Liste_der_Informationstafeln_an_Denkmalschutz_Objekten?format=geojson_link')

## Analyze the data

In [ ]:
gdf.plot()

In [ ]:
# drop columns that have no values
gdf.dropna(how='all', axis=1, inplace=True)

In [ ]:
print(f'The dataset has {gdf.shape[0]:,.0f} rows (observations) and {gdf.shape[1]:,.0f} columns (variables).')
print(f'There seem to be {gdf.duplicated().sum()} exact duplicates in the data.')

In [ ]:
gdf.info(memory_usage='deep', verbose=True)

In [ ]:
gdf.head()

In [ ]:
# display a small random sample transposed in order to see all variables
gdf.sample(3).T

In [ ]:
# describe non-numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(exclude='number'))
except:
    print("No categorical data in dataset.")

In [ ]:
# describe numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(include='number'))
except:
    print("No numercial data in dataset.")

In [ ]:
# plot a histogram for each numerical feature
try:
    gdf.hist(bins=25, rwidth=.9)
    plt.tight_layout()
    plt.show()
except:
    print("No numercial data to plot.") 

In [ ]:
# continue your code here...

**Contact**: opendata@zuerich.ch